In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_csv('anomaly.csv')
df.head()

,Date,Power,Detector,Quality
0,1-Jan-16,96,8,Good
1,2-Jan-16,96,10,Good
2,3-Jan-16,91,8,Good
3,4-Jan-16,97,9,Good
4,5-Jan-16,91,11,Good


In [3]:
print(df.columns) 
print(df.head())
#df['Power'].plot(kind='hist', title='Power Reading', bins=30, figsize=(12,10)) 
#Most values between 90 and 100 with some outliers / anomalies
#df['Detector'].plot(kind='hist', title='Detector Reading', bins=30, figsize=(12,10)) 
#Most values between 8 and 12 with some outliers / anomalies

#To see how the data is spread betwen Good and Bad
print(df.groupby('Quality')['Quality'].count())

Index(['Date', 'Power', 'Detector', 'Quality'], dtype='object')
       Date  Power  Detector Quality
0  1-Jan-16     96         8    Good
1  2-Jan-16     96        10    Good
2  3-Jan-16     91         8    Good
3  4-Jan-16     97         9    Good
4  5-Jan-16     91        11    Good
Quality
Bad      407
Good    1054
Name: Quality, dtype: int64


In [5]:
df.drop(['Date'], axis=1, inplace=True)
df.head()

,Power,Detector,Quality
0,96,8,Good
1,96,10,Good
2,91,8,Good
3,97,9,Good
4,91,11,Good


In [6]:
#If there are missing entries, drop them.
df.dropna(inplace=True,axis=1)


#COnvert non-numeric to numeric
df.Quality[df.Quality == 'Good'] = 1
df.Quality[df.Quality == 'Bad'] = 2


good_mask = df['Quality']== 1 #All good to be True for good data points
bad_mask = df['Quality']== 2 #All values False for good data points
#print(good_mask.head())

C:\Users\Mahdi\AppData\Local\Temp/ipykernel_5424/4211872558.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Good'] = 1
C:\Users\Mahdi\AppData\Local\Temp/ipykernel_5424/4211872558.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Bad'] = 2


In [7]:
df.drop('Quality',axis=1,inplace=True)

df_good = df[good_mask]
df_bad = df[bad_mask]

print(f"Good count: {len(df_good)}")
print(f"Bad count: {len(df_bad)}")

Good count: 1054
Bad count: 407


In [8]:
# This is the feature vector that goes to the neural net
x_good = df_good.values
x_bad = df_bad.values



x_good_train, x_good_test = train_test_split(
        x_good, test_size=0.25, random_state=42)

print(f"Good train count: {len(x_good_train)}")
print(f"Good test count: {len(x_good_test)}")

Good train count: 790
Good test count: 264


In [9]:
model = Sequential()
model.add(Dense(10, input_dim=x_good.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(x_good.shape[1])) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
 dense_2 (Dense)             (None, 10)                40        
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(x_good_train,x_good_train,verbose=1,epochs=100)

pred = model.predict(x_good_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_good_test))

pred = model.predict(x_good)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_good))

pred = model.predict(x_bad)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_bad))

Epoch 1/100
25/25 [==============================] - 1s 2ms/step - loss: 4906.5220
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 4561.7612
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 4558.3574
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 4554.5933
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 4550.5010
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 4546.0757
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 4541.3140
Epoch 8/100
25/25 [==============================] - 0s 2ms/step - loss: 4536.2202
Epoch 9/100
25/25 [==============================] - 0s 2ms/step - loss: 4530.7935
Epoch 10/100
25/25 [==============================] - 0s 2ms/step - loss: 4525.0342
Epoch 11/100
25/25 [==============================] - 0s 2ms/step - loss: 4518.9502
Epoch 12/100
25/25 [==============================] - 0s 2ms/step - loss: 4512.5381
E

In [11]:
print(f"Insample Good Score (RMSE): {score1}".format(score1))
print(f"Out of Sample Good Score (RMSE): {score2}")
print(f"Bad sample Score (RMSE): {score3}")

Insample Good Score (RMSE): 56.4165839549305
Out of Sample Good Score (RMSE): 56.336939935176396
Bad sample Score (RMSE): 59.09162390122287
